In [ ]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from moviepy.editor import VideoFileClip
from collections import deque
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [ ]:
'''Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.'''
object_points=[]
image_points=[]
objp = np.zeros((6*9,3), np.float32) # 6,9 represents the total number of co-ordinates in y and x direction respectively.
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# Iterate through the lists and get the corners of the images
#plt.figure(figsize = (16,32)) # To plot the (width, height) in inches
for image in os.listdir("camera_cal/"): # image will iterate through the list of the paths and consider each path one after another 
    img = cv2.imread("camera_cal/" + image) # img stores the data of the image located in the path in the form of numpy array and reads the data of pixel in (BGR) format.    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # gray stres the graysacle img
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None) # Prints the 9*6=54 corners (x,y) co-ordinates.
    # if all the corners are present in the gray(image) the image returns True
    if ret == True :
        object_points.append(objp) #objp are added to the object_points
        image_points.append(corners) # Corners are added to the image_points 
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)

In [ ]:
def threshold_binary_image(img):    


    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Sobel x and Sobel y
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 15) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx)) #Rescale to unit 8
    sobely = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 15) # Take the derivative in y
    abs_sobely = np.absolute(sobely) # Absolute y derivative to accentuate lines away from horizontal
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely)) #Rescale to unit 8

    # Threshold x gradient and y gradient
    thresh_min = 50
    thresh_max = 150
    sxbinary = np.zeros_like(scaled_sobelx) # Has similar dimensions to the scaled_sobelx but replcaes elements with 0.
    sxbinary[(scaled_sobelx >= 35) & (scaled_sobelx <= thresh_max)] = 1 # Making the points which fall in threshold to 1.
    sybinary = np.zeros_like(scaled_sobely)
    sybinary[(scaled_sobely >= thresh_min) & (scaled_sobely <= thresh_max)] = 1
    
    # Magnitude of gradient
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    scaled_sobelm = np.uint8(255*magnitude/np.max(magnitude)) 
    smbinary = np.zeros_like(magnitude) 
    smbinary[(scaled_sobelm >= thresh_min) & (scaled_sobelm <= thresh_max)] = 1 # Create a binary image of ones where threshold is met, zeros otherwise
    
    # Direction of the gradient
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # By increasing the size of kenrel we were able to pick up particular points
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1) # So I have taken the kernel sixe = 31 which is maximum in this case.
    direction = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    sdbinary =  np.zeros_like(direction)
    sdbinary[(direction >= 0.7) & (direction <= 1.0)] = 1
    
    # Compute the combination of Sobel X and Sobel Y or Magnitude and Direction
    sxymd_binary = np.zeros_like(sdbinary)
    sxymd_binary[(sxbinary == 1) | ((sybinary == 1) & (smbinary == 1) & (sdbinary == 1))] = 1
    
    

    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    yellow = np.zeros_like(hls[:,:,0])
    yellow[((hls[:,:,0] >= 15) & (hls[:,:,0] <= 35)) & ((hls[:,:,1] >= 30) & (hls[:,:,1] <= 204)) & ((hls[:,:,2] >= 115) & (hls[:,:,2] <= 255))] = 1
    
    white = np.zeros_like(hls[:,:,0])
    white[((hls[:,:,0] >= 0) & (hls[:,:,0] <= 255)) & ((hls[:,:,1] >= 200) & (hls[:,:,1] <= 255)) & ((hls[:,:,2] >= 0) & (hls[:,:,2] <= 255))] = 1               
    
    combined_binary = np.zeros_like(sxymd_binary)
    combined_binary[(white == 1) & (sxymd_binary == 1)] = 1
    
    binary = np.zeros_like(sxymd_binary)
    binary[(combined_binary == 1) | (yellow == 1)] = 1
    
    return binary
    

In [ ]:
def perspective_transform(img):
    
    width = img.shape[1] # Width of the image
    height = img.shape[0] # Height of the image
    # Points selected on trial and error basis
    src = np.array([[210,height],[595,450],[690,450], [1110, height]], dtype=np.float32)
    offset = 200
    dst = np.array([[offset, height],[offset, 0],[width-offset, 0], [width-offset, height]], dtype=np.float32)
    # Perspective transform
    M = cv2.getPerspectiveTransform(src, dst)
    # Inverse perspective transform
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warped output
    warped = cv2.warpPerspective(img, M, (width, height), flags=cv2.INTER_LINEAR)
    return Minv, warped
    
    

In [ ]:
LEFT = Line()
RIGHT = Line()

def sliding_windows(binary_warped):
    
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    nwindows = 9
    window_height = np.int(binary_warped.shape[0]/nwindows)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base
    margin = 50
    minpix = 1
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [0, 255, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 255, 0]
    
    #For Video:
    LEFT.recent_xfits = [(left_fitx)]
    LEFT.best_xfit = np.mean(LEFT.recent_xfits)
    RIGHT.recent_xfits = [(right_fitx)]
    RIGHT.best_xfit = np.mean(RIGHT.recent_xfits)
        
    LEFT.current_fit_coeffs = [(left_fit)]
    LEFT.best_fit_coeffs = np.mean(LEFT.current_fit_coeffs)
    RIGHT.current_fit_coeffs = [(right_fit)]
    RIGHT.best_fit_coeffs = np.mean(RIGHT.current_fit_coeffs)
    
    LEFT.line_base_position = leftx_current
    RIGHT.line_base_position = rightx_current
    
    LEFT.allx = leftx
    LEFT.ally = lefty
    RIGHT.allx = rightx
    RIGHT.ally = righty
    
    return out_img, ploty, left_fitx, right_fitx, left_fit, right_fit, leftx, rightx, leftx_base, rightx_base


pipeline(op='Sliding Windows')